In [1]:
# Import secrets file
from secrets import client_id, client_secret, user_id

# Import Libraries
import os
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
!pip install --default-timeout=100 future

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Connect to the API

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

In [3]:
def get_playlist(user, playlist_id, filename):
    # Retrieve Track IDs
    r = sp.user_playlist_tracks(user,playlist_id)
    t = r['items']
    ids = []
    while r['next']:
        r = sp.next(r)
        t.extend(r['items'])
    for s in t: ids.append(s["track"]["id"])

    # Grab track info from the track URIs
    def getTrackFeatures(id):
        meta = sp.track(id)
        features = sp.audio_features(id)
        # meta
        name = meta['name']
        album = meta['album']['name']
        artist = meta['album']['artists'][0]['name']
        release_date = meta['album']['release_date']
        length = meta['duration_ms']
        popularity = meta['popularity']
        # features
        acousticness = features[0]['acousticness']
        danceability = features[0]['danceability']
        valence = features[0]['valence']
        key = features[0]['key']
        energy = features[0]['energy']
        instrumentalness = features[0]['instrumentalness']
        liveness = features[0]['liveness']
        loudness = features[0]['loudness']
        speechiness = features[0]['speechiness']
        mode = features[0]['mode']
        tempo = features[0]['tempo']
        time_signature = features[0]['time_signature']

        track = [name, album, artist, release_date, length, 
                 popularity, acousticness, danceability, 
                 valence, key, energy, instrumentalness, liveness, loudness, 
                 speechiness, mode, tempo, time_signature]
        
        return track

    # Loop over track IDs
    tracks = []
    for i in range(len(ids)):
      time.sleep(.5)
      track = getTrackFeatures(ids[i])
      tracks.append(track)

    # Create dataset
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 
                                         'popularity', 'acousticness', 'danceability', 'valence', 
                                         'key', 'energy', 'instrumentalness', 'liveness', 'loudness', 
                                         'speechiness', 'mode', 'tempo', 'time_signature'])
    df.to_csv(f"{filename}.csv", sep = ',', index=False)

    # Moving the file to a folder
    if not os.path.exists("data"):
      os.mkdir("data")
    os.replace(f"{filename}.csv", f"data/{filename}.csv")

In [4]:
get_playlist(user_id, '4u2bO89cFY7W6n1p2EWY0K', 'my_playlist')

In [5]:
my_playlist = pd.read_csv("data/my_playlist.csv")
my_playlist

,name,album,artist,release_date,length,popularity,acousticness,danceability,valence,key,energy,instrumentalness,liveness,loudness,speechiness,mode,tempo,time_signature
0,Getting There,Until The Quiet Comes,Flying Lotus,2012-10-01,109353,40,0.1490,0.180,0.100,1,0.719,0.00037,0.1310,-5.789,0.0993,1,163.801,4
1,Massage Situation,Reset EP,Flying Lotus,2007-10-01,167173,46,0.0108,0.603,0.403,3,0.606,0.00157,0.1080,-5.060,0.0688,0,93.060,4
2,Do The Astral Plane,Cosmogramma,Flying Lotus,2010-05-03,237920,45,0.0505,0.722,0.275,8,0.828,0.00000,0.1140,-3.227,0.1900,0,117.693,4
3,Camel,Los Angeles,Flying Lotus,2008-07-09,142893,35,0.0132,0.699,0.584,0,0.926,0.20000,0.3110,-6.142,0.2850,1,84.863,4
4,Putty Boy Strut,Until The Quiet Comes,Flying Lotus,2012-10-01,173687,33,0.1030,0.723,0.711,1,0.432,0.21800,0.0883,-7.891,0.4180,1,163.973,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Oh How Lovely (feat. Tomppabeats),For Her,wüsh,2016-11-04,80308,44,0.2480,0.793,0.407,3,0.412,0.00000,0.5370,-10.517,0.2100,1,77.729,4
200,Space Cowboy,Ipanema,Saib,2015-04-28,152625,47,0.4740,0.854,0.722,0,0.689,0.81600,0.1530,-15.050,0.1700,1,97.993,4
201,m i s t,ep seeds,eevee,2017-03-16,118182,57,0.3540,0.624,0.395,1,0.229,0.47000,0.1160,-15.698,0.0460,0,130.069,4
202,Football Head,Pleasure Palette,Flamingosis,2016-06-10,224916,22,0.6600,0.662,0.584,1,0.212,0.57000,0.6090,-14.638,0.0523,1,89.996,4


In [6]:
playlist_names = ['lofi_beats', 'music_for_plants', 'indie_sunshine', 'lowkey', 'golden_hour', 'lofi_hip_hop', 
                  'retrowave_outrun', 'chill_lofi_study_beats', 'electronica_romantica', 'brain_food', 
                  'jazz_vibes', 'metaverses', 'chill_vibes', 'gold_instrumental_beats', 'the_lofi_garden', 
                  'alternative_hip_hop', 'alternative_rnb', 'floating', 'serotonin', 'poolside_disco', 
                  'french_touch', 'park_hangs', 'pollen', 'modern_psychedelia', 'fit_check', 'nu_funk', 
                  'robo_funk', 'queen', 'levitate']

playlist_uris = ['37i9dQZF1DWWQRwui0ExPn', '37i9dQZF1DXclWedfNUp3z', '37i9dQZF1DWUWC0NIJDJKL',
                 '37i9dQZF1DX2yvmlOdMYzV', '37i9dQZF1DWUE76cNNotSg', '37i9dQZF1DX36Xw4IJIVKA',
                 '37i9dQZF1DXdLEN7aqioXM', '37i9dQZF1DX8Uebhn9wzrS', '37i9dQZF1DXaRycgyh6kXP',
                 '37i9dQZF1DWXLeA8Omikj7', '37i9dQZF1DX0SM0LYsmbMT', '37i9dQZF1DX0Uh73O0FmIK',
                 '37i9dQZF1DX889U0CL85jj', '37i9dQZF1DX0dU0Z2LOkZN', '37i9dQZF1DX4t95PAs1EpY',
                 '37i9dQZF1DWTggY0yqBxES', '37i9dQZF1DWSfMe9z89s9B', '37i9dQZF1DXdEZt3tTR001',
                 '37i9dQZF1DWYMroOc5KTTh', '37i9dQZF1DWSvfPiFfb8Mi', '37i9dQZF1DX9cbNxuNYT3d',
                 '37i9dQZF1DWSP55jZj2ES3', '37i9dQZF1DWWBHeXOYZf74', '37i9dQZF1DX8gDIpdqp1XJ',
                 '37i9dQZF1DX1dvMSwf27JO', '37i9dQZF1DWZgauS5j6pMv', '37i9dQZF1DWUcRrhkfhG22',
                 '37i9dQZF1DWSIO2QWRavWZ', '37i9dQZF1DWVY5eNJoKHd2']

recommended_playlists = dict(zip(playlist_uris, playlist_names))
recommended_playlists

{'37i9dQZF1DWSIO2QWRavWZ': 'queen',
 '37i9dQZF1DWSP55jZj2ES3': 'park_hangs',
 '37i9dQZF1DWSfMe9z89s9B': 'alternative_rnb',
 '37i9dQZF1DWSvfPiFfb8Mi': 'poolside_disco',
 '37i9dQZF1DWTggY0yqBxES': 'alternative_hip_hop',
 '37i9dQZF1DWUE76cNNotSg': 'golden_hour',
 '37i9dQZF1DWUWC0NIJDJKL': 'indie_sunshine',
 '37i9dQZF1DWUcRrhkfhG22': 'robo_funk',
 '37i9dQZF1DWVY5eNJoKHd2': 'levitate',
 '37i9dQZF1DWWBHeXOYZf74': 'pollen',
 '37i9dQZF1DWWQRwui0ExPn': 'lofi_beats',
 '37i9dQZF1DWXLeA8Omikj7': 'brain_food',
 '37i9dQZF1DWYMroOc5KTTh': 'serotonin',
 '37i9dQZF1DWZgauS5j6pMv': 'nu_funk',
 '37i9dQZF1DX0SM0LYsmbMT': 'jazz_vibes',
 '37i9dQZF1DX0Uh73O0FmIK': 'metaverses',
 '37i9dQZF1DX0dU0Z2LOkZN': 'gold_instrumental_beats',
 '37i9dQZF1DX1dvMSwf27JO': 'fit_check',
 '37i9dQZF1DX2yvmlOdMYzV': 'lowkey',
 '37i9dQZF1DX36Xw4IJIVKA': 'lofi_hip_hop',
 '37i9dQZF1DX4t95PAs1EpY': 'the_lofi_garden',
 '37i9dQZF1DX889U0CL85jj': 'chill_vibes',
 '37i9dQZF1DX8Uebhn9wzrS': 'chill_lofi_study_beats',
 '37i9dQZF1DX8gDIpdqp1

In [7]:
for uri, name in recommended_playlists.items():
  try:
    get_playlist(user_id,uri,name)
  except TypeError:
    continue